In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb


In [20]:
# Criando função para calcular as métricas
def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }

In [21]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    elif scale == 'YcrCb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    else:
        raise ValueError("Escala de cor não suportada.")

In [22]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor
def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário

                # Converta para a escala de cor desejada
                img = convert_color_scale(img, color_scale)

                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal

                images.append(img)
    return np.array(images)

In [23]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
normal_dir = os.path.join(data_dir, 'Non_Cancerous')
cancer_dir = os.path.join(data_dir, 'Serous')
img_extensions = ['.jpg', '.jpeg', '.png']


In [ ]:
model_mapping = {
    'VGG16': VGG16,
    'ResNet50': ResNet50,
    'InceptionV3': InceptionV3,
    'EfficientNetB0': EfficientNetB0,
    'MobileNetV2': MobileNetV2
}


tds_Cnns = ['VGG16', 'ResNet50', 'InceptionV3', 'EfficientNetB0', 'MobileNetV2']
df_all_metrics = pd.DataFrame()
for CNN in tds_Cnns:

    # Carregar o modelo InceptionV3 pré-treinado
    base_model = model_mapping[CNN](weights='imagenet', include_top=False)     # <----------------------------------------------------------

    # Extrair características usando o modelo VGG16
    def extract_features(model, X):
        features = model.predict(X)
        return features.reshape((features.shape[0], -1))


    #--------------------------------------------------------------------------------------------------------------------------------------

    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(normal_dir, 'grayscale',img_extensions)        # <----------------------------------------------------------
    cancer_images = load_images(cancer_dir, 'grayscale',img_extensions)        # hsv<----------------------------------------------------------


    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_images = extract_features(base_model, all_images)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=41)

    X_train = X_train / 255.0
    X_test = X_test / 255.0
    #--------------------------------------------------------------------------------------------------------------------------------------



    # Para o SVM
    svm_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    metrics_svm = get_metrics(y_test, y_pred_svm)
    print(f"Métricas para {CNN}: (Com RF)")
    print(metrics_svm)



    # Criar DataFrames e salvar os resultados em um xlsx 
    df_metrics_svm = pd.DataFrame([metrics_svm.values()], columns=metrics_svm.keys())
    df_metrics_svm.insert(0, 'CNN', CNN)




    # Concatenar todos os DataFrames
    df_all_metrics = pd.concat([df_all_metrics,df_metrics_svm], ignore_index=True)

# Salvar o DataFrame em um arquivo Excel
df_all_metrics.to_excel('ResultaadoCNN-RF.xlsx', index=False)  # <----------------------------------------------------------

df_all_metrics



Métricas para VGG16: (Com RF)
{'accuracy': 0.85, 'specificity': 0.8, 'recall': 0.9, 'precision': 0.8181818181818182, 'f1-score': 0.8571428571428572, 'kappa': 0.7, 'auc-roc': 0.85}
Métricas para ResNet50: (Com RF)
{'accuracy': 0.85, 'specificity': 0.85, 'recall': 0.85, 'precision': 0.85, 'f1-score': 0.85, 'kappa': 0.7, 'auc-roc': 0.85}
Métricas para InceptionV3: (Com RF)
{'accuracy': 0.725, 'specificity': 0.9, 'recall': 0.55, 'precision': 0.8461538461538461, 'f1-score': 0.6666666666666667, 'kappa': 0.44999999999999996, 'auc-roc': 0.725}
Métricas para EfficientNetB0: (Com RF)
{'accuracy': 0.85, 'specificity': 0.9, 'recall': 0.8, 'precision': 0.8888888888888888, 'f1-score': 0.8421052631578948, 'kappa': 0.7, 'auc-roc': 0.8500000000000001}
Métricas para MobileNetV2: (Com RF)
{'accuracy': 0.7, 'specificity': 0.65, 'recall': 0.75, 'precision': 0.6818181818181818, 'f1-score': 0.7142857142857143, 'kappa': 0.4, 'auc-roc': 0.7}


,CNN,accuracy,specificity,recall,precision,f1-score,kappa,auc-roc
0,VGG16,0.850,0.80,0.90,0.818182,0.857143,0.70,0.850
1,ResNet50,0.850,0.85,0.85,0.850000,0.850000,0.70,0.850
2,InceptionV3,0.725,0.90,0.55,0.846154,0.666667,0.45,0.725
3,EfficientNetB0,0.850,0.90,0.80,0.888889,0.842105,0.70,0.850
4,MobileNetV2,0.700,0.65,0.75,0.681818,0.714286,0.40,0.700
